In [1]:
from mutagen.mp4 import MP4
from pathlib import Path
import re

In [2]:
input_path = Path.cwd() / 'input'

In [3]:
input_folder = 'Watch - TV Shows (Anime)'
input_path = Path('f:/') / input_folder

In [4]:
folder_list = [path for path in input_path.iterdir() if path.is_dir()]

In [5]:
def update_title(file_path):
    file = MP4(file_path)
    title = ''.join(file['©nam'])
    
    if not title:
        file['©nam'] = file_path.stem
        file['©cmt'] = file_path.stem
        file.save()

In [6]:
def get_episode(file_path):
    patterns = [
        r'\[E\d+\]',
        r'\[\d+(\.\d)?\]',
        r'\[\d+(\.\d)?v\d\]',
        r'\[\d+\s?END\]',
        r'\[\d+_\d+\]',
        r'-\s?\d+',
        r'\.\d+',
        r'第\d+话',
        r'第\d+話'
    ]
    
    for pattern in patterns:
        result = re.search(pattern, file_path.stem)
        if result:
            text = result.group()
            search = re.search(r'\d+(\.\d)?', text)
            episode = search.group()
            return episode

In [7]:
for folder_path in folder_list:
    file_list = list(folder_path.glob('*.mp4'))
    
    episodes = []
    for file_path in file_list:
        update_title(file_path)
        episode = get_episode(file_path)
        if not episode:
            print(file_path.stem)
        episodes.append(episode)
    
    num_digit = max([len(x.split('.')[0]) for x in episodes])
    episodes = [x.zfill(num_digit) for x in episodes]
    
    print(folder_path.name)
    print(episodes)
    print()

[86-Eight_Six][86-不存在的战区][01-23][CHS][1080P][MP4]
['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11.5', '11', '12', '13', '14', '15', '16', '17', '18.5', '18', '19', '20', '21', '22', '23']

[Ace_of_Diamond][钻石王牌][S1][01-75][CHS][720P][MP4]
['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75']

[Ace_of_Diamond][钻石王牌][S2][01-51][CHS][720P][MP4]
['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '